In [1]:
#!/usr/bin/python
# Import packages
import numpy as np
import tensorflow as tf
import sys
import json
import paho.mqtt.client as paho
import os
import socket
import ssl
from time import sleep
from random import uniform
import time
import datetime
import os
import cv2
from random import uniform
import time
import datetime

### Real time

In [2]:
count = 0
connflag = False

In [3]:
#Defination for connection
def on_connect(client, userdata, flags, rc):
    global connflag
    connflag = True
    print("Connection returned result: " + str(rc) )

def on_message(client, userdata, msg):
    print(msg.topic+" "+str(msg.payload))

In [4]:
#Init obs for paho mqtt 
mqttc = paho.Client()
mqttc.on_connect = on_connect
mqttc.on_message = on_message

In [5]:
#Define host and port no
host = "192.168.0.103"
port = 1883

In [6]:
#connect to broker
mqttc.connect(host, port, keepalive=60)
mqttc.loop_start()

Connection returned result: 0


In [7]:
# init face harcascade
clas = cv2.CascadeClassifier('C:/Users/sarth/Anaconda projects/self projects and others/VGG face/25. Face Recognition/Haarcascades/haarcascade_frontalface_default.xml')


## Final

In [8]:
#Define variables
thresh1 = 250
thresh2 = 400
control = 5  #For division
control2 = 8 #for sending data
control3 = 25 #for invoking non detection

count = 0 
count1 = 0
count2 = 0
val2 = 0
val = 90
temp = 90
cou = True

In [ ]:
#Open Webcam
cap = cv2.VideoCapture(1) 
while(True):
    
    ret, img = cap.read()
    faces2 = clas.detectMultiScale(img, 1.3, 5)
    
    for (x,y,w,h) in faces2:
        if w > 80:
            #circle(frame, Point2i(x-coordinate, y-coordinate), 5, Scalar(0,125,230), 4, 3);
            cv2.line(img, (thresh1, 0), (thresh1, 480), (10,90,200), 1)
            cv2.line(img, (thresh2, 0), (thresh2, 480), (10,90,200), 1)
            
            cv2.rectangle(img,(x,y),(x+w,y+h),(105,200,100),3) 
            
            x1 = int(x+w/2)-2
            y1 = int(y+h/2)-2
            
            cv2.rectangle(img,(x1,y1),(x1+2,y1+2),(105,200,100),3)
            cv2.rectangle(img,(0,240),(0+10,240+20),(140,150,230),5)
            cv2.rectangle(img,(640,255),(640-10,255-20),(140,150,230),5)
            
            #print(x1)       
            if x1 < thresh1: ###thresh1 = 180
                cv2.putText(img, "move left", (155,50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2,(120,20,255),2)
                val = round((thresh1 - x1)/control)
                val = abs(temp + val)
                
                if (connflag == True) and (count >= control2):
                    tempreading = uniform(20.0,25.0)
                    mqttc.publish("Status", val, qos=1)
                    count = 0
                    temp = val


                
            elif x1 > thresh2: ###thresh2 = 460
                cv2.putText(img, "move right", (155,50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2,(120,20,255),2)
                val = round((thresh2 - x1)/control)
                val = abs(temp + val)
                if (connflag == True) and (count >= control2):
                    tempreading = uniform(20.0,25.0)
                    mqttc.publish("Status", val, qos=1)
                    count = 0
                    temp = val

                                    

            count += 1
            count2 = 0
            val2 = 0
        #count2 = 0
            

    if (faces2 == ()) and (count2 >= control3):
        cv2.putText(img, "No Face", (185,50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 3,(80,130,230),3)
        
        if cou:
            cv2.putText(img, "Self Rotation Activated", (55,150), cv2.FONT_HERSHEY_COMPLEX_SMALL , 2, (10,250,30), 3, lineType=True)
            cou = False
        
        elif not cou:
            cv2.putText(img, "", (55,150), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2,(90,200,60),3, lineType=True)
            cou = True

        temp = 90
        temp2 = 90
        if (connflag == True) and (count1 >= 70):
            if val2 == 90:
                val2 = 40
                    
            elif val2 == 40:
                val2 = 130
                
            elif val2 == 0 or val2 == 130:
                val2 = 90
                
            tempreading = uniform(20.0,25.0)
            mqttc.publish("Status", val2, qos=1)
            count1 = 0
                
        count1 += 1
        
    count2 += 1

    
    cv2.imshow('img', img)
    
    

    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()


C:\Users\sarth\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


## NOTE 